In [1]:
import json
import os
from typing import Dict, Any


class PDFParser:
    def parse(
        self, template: Dict[str, Any], pdf_data: Dict[str, Any]
    ) -> Dict[str, Any]:
        pass


template_path: str = os.path.join("src", "templates", "barclays_template.json")
pdf_data_path: str = os.path.join("src", "pdf_data", "barclays_pdf_data.json")

template: Dict[str, Any] = json.load(open(template_path))
pdf_data: Dict[str, Any] = json.load(open(pdf_data_path))

parser: PDFParser = PDFParser()
output_generated: Dict[str, Any] = parser.parse(template, pdf_data)

output_data = json.load(open("src/outputs/barclays_apr_2_output.json"))


In [2]:
# Create a FormParser

In [3]:
from typing import List, Union

def page_number_converter(page_numbers: str, number_of_pages: int) -> List[int]:    
    if ":" in page_numbers:
        left_index = int(page_numbers.split(":")[0])
        right_index = int(page_numbers.split(":")[1])
    else:
        index = int(page_numbers)
        if index >= 0:
            index = index - 1
        elif index < 0:
            index = number_of_pages + index
        return [index]

    if left_index > 0:
        left_index -= 1
    if right_index > 0:
        right_index -= 1

    if left_index < 0:
        left_index = number_of_pages + left_index + 1
    if right_index < 0:
        right_index = number_of_pages + right_index + 1

    if left_index == right_index:
        return [left_index]
    
    return list(range(left_index, right_index))


In [4]:
from src.parser import FormParser

for page_rule in template["pages"]:
    page_indexes = page_number_converter(page_rule["page_numbers"], len(pdf_data["pages"]))
    for page_index in page_indexes:
        if "forms" in page_rule and len(page_rule["forms"]) > 0:
            for rule_id in page_rule["forms"]:
                try:
                    output_data_for_rule = FormParser().get_output_data_for_rule(rule_id, page_index, pdf_data, template)
                    print(output_data_for_rule)
                except IndexError:
                    print(f"Rule ID '{rule_id}' not found in template rules or page index '{page_index}' is out of range.")


{'sort_code': '20-44-51'}
{'account_number': '13415759'}
{'customer_name': 'Miss Meghan Victoria Thorneloe'}
{'start_balance': '£73.72'}
{'money_in': ''}
{'money_out': ''}
{'end_balance': ''}
{'overdraft_limit': ''}


In [5]:
from src.parser import FormParser

page_index = 1
output_data_for_rule = FormParser().get_output_data_for_rule("start_balance", page_index, pdf_data, template)

In [6]:
output_data_for_rule

{'start_balance': '£73.72'}

In [7]:
page_data = pdf_data["pages"][page_index]["content"]

rule = template["rules"][1]

print(rule)
coordinates = rule["config"]["coordinates"]
FormParser().get_items_in_bounding_box(page_data, coordinates)

{'rule_id': 'account_number', 'type': 'form', 'config': {'field_name': 'account_number', 'coordinates': {'top_left': {'x': 0.81, 'y': 0.115}, 'bottom_right': {'x': 0.88, 'y': 0.128}}, 'type': 'text'}}


[{'text': '13415759',
  'bounding_box': {'coordinates': {'top_left': {'x': 484.75, 'y': 99.0},
    'bottom_right': {'x': 520.4, 'y': 107.0}},
   'decimal_coordinates': {'top_left': {'x': 0.814706, 'y': 0.117577},
    'bottom_right': {'x': 0.874622, 'y': 0.127078}}}}]